# Length

In [784]:
import os
import pandas
import numpy as np
import pandas as pd
import re
import ast
import json
from CS_feature_extractor import *
DATA_DIR = '../data'

# Experiment Config
DF_NAME = 'GSM8K'

NUM_OF_SAMPLES = 500
NUM_OF_COT = 40
MODEL = 'gpt-3.5-turbo-0125'
storage_dir = os.path.join(DATA_DIR, f'Evaluation_CoTs/Algo_Design_Data')
hard_file_path = os.path.join(storage_dir, f'{DF_NAME}_hard.json')
easy_file_path = os.path.join(storage_dir, f'{DF_NAME}_easy.json')

In [785]:
def sigmoid(x):
    return 1 / (1 + np.exp(-(x-5)))

In [786]:
with open(hard_file_path,'r') as f:
    data = json.load(f)
hard = pd.DataFrame(data)
hard['size_of_cot'] = [[(i+1)^2 for i in range(40)] for j in range(500)]
for row_idx in range(len(hard)):
    hard['SIM_COT_AGG'][row_idx][0] = 0.5
# hard['SIM_COT_AGG'] = hard['SIM_COT_AGG'].apply(lambda lst: [x*sigmoid(idx) for idx,x in enumerate(lst)])
hard

,id,correct answer,CoT answers,Correctness,LEN,QUA_IM,DIF_IV,DIF_SUB,SIM_COT_BIGRAM,SIM_COT_AGG,SIM_COT_PW,SIM_AC_BIGRAM,SIM_AC_AGG,SIM_AC_PW,size_of_cot
0,0,-9867630,"[0, 9867650, 19735200, -9867630, 1973528, 1973...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 5, 5, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Challenging Math,"[0.0, 0.8827160493827161, 0.9006211180124224, ...","[0.5, 0.8827160493827161, 0.8963414634146342, ...","[0.0, 0.8827160493827161, 0.9207052958483164, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
1,1,3431580,"[3431580, 3431580, 3431580, 3431580, 3431580, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[4, 4, 4, 4, 4, 4, 4, 3, 2, 4, 4, 4, 4, 2, 2, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, ...",Challenging Math,"[0.0, 0.8947368421052632, 0.6111111111111112, ...","[0.5, 0.8947368421052632, 0.6027397260273972, ...","[0.0, 0.8947368421052632, 0.6339137645107795, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
2,2,322886700,"[14, 14, 14, 107628900, 107628900, 14, 1076289...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, ...",Challenging Math,"[0.0, 0.9117647058823529, 1.0, 0.8676470588235...","[0.5, 0.9117647058823529, 0.948905109489051, 0...","[0.0, 0.9117647058823529, 0.9558823529411764, ...","[0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, ...","[0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, ...","[0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
3,3,-6887448,"[6887423, 6887423, 6887398, 6887398, 167, 206,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4, 3, 3, 3, 3, 4, 5, 5, 3, 3, 5, 0, 3, 3, 3, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 5, 0, 5, 2, 5, 5, 2, 9, 10, 5, 9, 2,...",Challenging Math,"[0.0, 0.874251497005988, 0.6787564766839378, 1...","[0.5, 0.874251497005988, 0.7040816326530612, 0...","[0.0, 0.874251497005988, 0.707657808234442, 0....","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
4,4,21459061,"[21459067, 21459067, 16104295, 21459060, 16119...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[6, 4, 1, 6, 5, 6, 5, 5, 4, 5, 1, 6, 3, 6, 5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Challenging Math,"[0.0, 0.6701570680628273, 0.4605263157894737, ...","[0.5, 0.6701570680628273, 0.4595744680851064, ...","[0.0, 0.6701570680628273, 0.41422542204568025,...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,100,"[80, 40, 100, 100, 100, 100, 100, 100, 100, 10...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 2, 0, 0, 0, 0, 9, 2, 2, 0, 0, 0, 0, 0, ...",Challenging Math,"[0.0, 0.7612903225806451, 0.6611111111111111, ...","[0.5, 0.7612903225806451, 0.6451612903225806, ...","[0.0, 0.7612903225806451, 0.6762345679012345, ...","[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 1

In [787]:
with open(easy_file_path,'r') as f:
    data = json.load(f)
easy = pd.DataFrame(data)
easy['size_of_cot'] = [[i+1 for i in range(40)] for j in range(500)]
for row_idx in range(len(easy)):
    easy['SIM_COT_AGG'][row_idx][0] = 0.5
easy['SIM_COT_AGG'] = easy['SIM_COT_AGG'].apply(lambda lst: [x*sigmoid(idx) for idx,x in enumerate(lst)])
easy

,id,correct answer,CoT answers,Correctness,LEN,QUA_IM,DIF_IV,DIF_SUB,SIM_COT_BIGRAM,SIM_COT_AGG,SIM_COT_PW,SIM_AC_BIGRAM,SIM_AC_AGG,SIM_AC_PW,size_of_cot
0,0,18,"[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0, 0, 5, 0, 0, ...",Math,"[0.0, 1.0, 0.7515527950310559, 0.6987951807228...","[0.0033464254621424277, 0.01786216713476679, 0...","[0.0, 1.0, 0.7515527950310559, 0.8462650602409...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,1,3,"[3, 3, 3, 5, 3, 5, 3, 3, 3, 45353, 3, 3, 3, 3,...","[1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, ...","[4, 3, 5, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, ...",Math,"[0.0, 0.7633136094674556, 0.740506329113924, 0...","[0.0033464254621424277, 0.013729118846803615, ...","[0.0, 0.7633136094674556, 0.7008087201125175, ...","[0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, ...","[0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, ...","[0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,2,70000,"[195000, 195000, 195000, 195000, 195000, 65000...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[4, 4, 4, 4, 4, 3, 8, 3, 8, 4, 3, 4, 4, 9, 3, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Math,"[0.0, 1.0, 0.8633540372670807, 0.8633540372670...","[0.0033464254621424277, 0.01798620996209156, 0...","[0.0, 1.0, 0.8633540372670807, 0.9544513457556...","[0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, ...","[0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,3,540,"[540, 540, 540, 540, 540, 540, 540, 540, 540, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Math,"[0.0, 1.0, 1.0, 1.0, 0.9782608695652174, 1.0, ...","[0.0033464254621424277, 0.01798620996209156, 0...","[0.0, 1.0, 1.0, 1.0, 0.9782608695652174, 0.982...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,4,20,"[140, 140, 35, 3, 760, 21, 20, 10, 35, 140, 35...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3, 3, 3, 4, 3, 3, 4, 5, 3, 3, 3, 3, 3, 4, 3, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Math,"[0.0, 0.984375, 0.7913669064748201, 0.66279069...","[0.0033464254621424277, 0.01770517543143388, 0...","[0.0, 0.984375, 0.7921120246659814, 0.63736150...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,200,"[200, 200, 200, 200, 2400, 2400, 2400, 2400, 2...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, ...","[3, 3, 2, 2, 3, 3, 3, 3, 3, 2, 2, 2, 6, 3, 3, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Math,"[0.0, 0.944954128440367, 0.9166666666666666, 1...","[0.0033464254621424277, 0.017161154459243322, ...","[0.0, 0.944954128440367, 0.9256230529595015, 0...","[0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, ...","[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 2, 3,

In [788]:
hard = pd.concat([hard,easy])
hard

,id,correct answer,CoT answers,Correctness,LEN,QUA_IM,DIF_IV,DIF_SUB,SIM_COT_BIGRAM,SIM_COT_AGG,SIM_COT_PW,SIM_AC_BIGRAM,SIM_AC_AGG,SIM_AC_PW,size_of_cot
0,0,-9867630,"[0, 9867650, 19735200, -9867630, 1973528, 1973...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 5, 5, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Challenging Math,"[0.0, 0.8827160493827161, 0.9006211180124224, ...","[0.5, 0.8827160493827161, 0.8963414634146342, ...","[0.0, 0.8827160493827161, 0.9207052958483164, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
1,1,3431580,"[3431580, 3431580, 3431580, 3431580, 3431580, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[4, 4, 4, 4, 4, 4, 4, 3, 2, 4, 4, 4, 4, 2, 2, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, ...",Challenging Math,"[0.0, 0.8947368421052632, 0.6111111111111112, ...","[0.5, 0.8947368421052632, 0.6027397260273972, ...","[0.0, 0.8947368421052632, 0.6339137645107795, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
2,2,322886700,"[14, 14, 14, 107628900, 107628900, 14, 1076289...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, ...",Challenging Math,"[0.0, 0.9117647058823529, 1.0, 0.8676470588235...","[0.5, 0.9117647058823529, 0.948905109489051, 0...","[0.0, 0.9117647058823529, 0.9558823529411764, ...","[0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, ...","[0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, ...","[0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
3,3,-6887448,"[6887423, 6887423, 6887398, 6887398, 167, 206,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4, 3, 3, 3, 3, 4, 5, 5, 3, 3, 5, 0, 3, 3, 3, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 5, 0, 5, 2, 5, 5, 2, 9, 10, 5, 9, 2,...",Challenging Math,"[0.0, 0.874251497005988, 0.6787564766839378, 1...","[0.5, 0.874251497005988, 0.7040816326530612, 0...","[0.0, 0.874251497005988, 0.707657808234442, 0....","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
4,4,21459061,"[21459067, 21459067, 16104295, 21459060, 16119...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[6, 4, 1, 6, 5, 6, 5, 5, 4, 5, 1, 6, 3, 6, 5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Challenging Math,"[0.0, 0.6701570680628273, 0.4605263157894737, ...","[0.5, 0.6701570680628273, 0.4595744680851064, ...","[0.0, 0.6701570680628273, 0.41422542204568025,...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,200,"[200, 200, 200, 200, 2400, 2400, 2400, 2400, 2...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, ...","[3, 3, 2, 2, 3, 3, 3, 3, 3, 2, 2, 2, 6, 3, 3, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Math,"[0.0, 0.944954128440367, 0.9166666666666666, 1...","[0.0033464254621424277, 0.017161154459243322, ...","[0.0, 0.944954128440367, 0.9256230529595015, 0...","[0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, ...","[0, 1, 1, 1, 0, 0, 0, 0, 0

# Early stopping and SC 

In [789]:
from collections import Counter

def calculate_correctness(df):
    # Define a helper function to determine the majority and compare it with the correct answer
    def check_majority(answers, correct):
        if not answers:
            return 0
        # Count the occurrences of each answer and find the most common one
        most_common = Counter(answers).most_common(1)[0][0]
        # Compare the most common answer with the correct answer
        return 1 if most_common == str(correct) else 0

    # Apply the helper function across the DataFrame rows
    df['SC_correctness'] = df.apply(lambda row: check_majority(row['CoT answers'], row['correct answer']), axis=1)
    return df

hard = calculate_correctness(hard)

In [790]:
hard.SC_correctness.value_counts()

SC_correctness
1    634
0    366
Name: count, dtype: int64

In [791]:
easy = calculate_correctness(easy)
easy.SC_correctness.value_counts()

SC_correctness
1    402
0     98
Name: count, dtype: int64

In [792]:
import pandas as pd

def evaluate_answers_with_window(df, window_size):
    # Helper function to evaluate answers with a sliding window
    def evaluate_window(answers, correct):
        # Track the number of steps (checks) taken
        steps = window_size-1
        
        # If the list is shorter than the window size, adjust the window size to the length of the list
        window_size_adjusted = min(window_size, len(answers))
        
        # Slide through the list with the adjusted window size
        for i in range(len(answers) - window_size_adjusted + 1):
            # Check the window content
            window = answers[i:i + window_size_adjusted]
            steps += 1
            
            # If all elements in the window are the same, evaluate correctness
            if window.count(window[0]) == window_size_adjusted:
                return 1 if window[0] == str(correct) else 0, steps
        
        # If no unanimous window is found, use the same value as SC_correctness and set steps to the length of the list
        majority = Counter(answers).most_common(1)[0][0]
        correctness = 1 if majority == str(correct) else 0
        return correctness, len(answers)

    # Apply the function to each row in the DataFrame
    result = df.apply(lambda row: evaluate_window(row['CoT answers'], row['correct answer']), axis=1)
    df['ES_correctness'] = result.apply(lambda x: x[0])
    df['ES_steps'] = result.apply(lambda x: x[1])

    return df

easy = evaluate_answers_with_window(easy,5)

In [793]:
easy.ES_correctness.value_counts()

ES_correctness
1    401
0     99
Name: count, dtype: int64

In [794]:
easy.ES_steps.mean()

13.15

In [795]:

hard = evaluate_answers_with_window(hard,5)
hard.ES_correctness.value_counts()

ES_correctness
1    634
0    366
Name: count, dtype: int64

In [796]:
hard.ES_steps.mean()

17.234

In [797]:
feature_li = [
 # 'LEN',
 'QUA_IM',
 'DIF_IV',
 # 'DIF_SUB',
 # 'SIM_COT_BIGRAM',
 'SIM_COT_AGG',
 # 'SIM_COT_PW',
 'SIM_AC_BIGRAM',
 'SIM_AC_AGG',
 'SIM_AC_PW',
    'size_of_cot'

 
  ]

# Visualize

In [798]:
import pandas as pd

def concatenate_columns(df,data_columns, outcome_column):
    # Initialize an empty dictionary to store the concatenated data
    concatenated_data = {}
    
    # Get the number of rows based on the length of the outcome column
    num_rows = len(df)
    
    # Iterate over each column
    for column in data_columns+[outcome_column]:
        # Initialize an empty list to store the concatenated values for the current column
        concatenated_values = []
        
        # Iterate over each row
        for i in range(num_rows):
            # Get the list of values for the current column and row
            
            values = df[column].iloc[i]
            
            # Concatenate the values into a single string
            concatenated_values += list(values)
            
        
        # Add the concatenated values to the dictionary with the column name as the key
        concatenated_data[column] = concatenated_values
        print(len(concatenated_values))
    # Add the outcome column to the concatenated data dictionary
    
    # Create a DataFrame from the concatenated data dictionary
    hard_final = pd.DataFrame(concatenated_data)
    
    return hard_final

hard_final = concatenate_columns(hard,feature_li,'Correctness')
easy_final = concatenate_columns(easy,feature_li,'Correctness')

40000
40000
40000
40000
40000
40000
40000
40000
20000
20000
20000
20000
20000
20000
20000
20000


In [799]:
hard_final['cot_answer'] = np.repeat(hard['correct answer'].values,40)

In [800]:
easy_final['cot_answer'] = np.repeat(easy['correct answer'].values,40)

In [801]:
from collections import Counter
print('--------------EASY--------------')
for col in easy_final.columns:
    if not col.startswith('SIM_COT_'):
        print(col, ':', Counter(easy_final[col]))
print('--------------HARD--------------')
for col in hard_final.columns:
    if not col.startswith('SIM_COT_'):
        print(col, ':', Counter(hard_final[col]))

--------------EASY--------------
QUA_IM : Counter({0: 19961, 1: 39})
DIF_IV : Counter({0: 18375, 2: 1127, 5: 279, 10: 124, 9: 83, 1: 7, 11: 3, 3: 1, 6: 1})
SIM_AC_BIGRAM : Counter({1: 14051, 0: 5949})
SIM_AC_AGG : Counter({1: 15238, 0: 4762})
SIM_AC_PW : Counter({1: 13897, 0: 6103})
size_of_cot : Counter({1: 500, 2: 500, 3: 500, 4: 500, 5: 500, 6: 500, 7: 500, 8: 500, 9: 500, 10: 500, 11: 500, 12: 500, 13: 500, 14: 500, 15: 500, 16: 500, 17: 500, 18: 500, 19: 500, 20: 500, 21: 500, 22: 500, 23: 500, 24: 500, 25: 500, 26: 500, 27: 500, 28: 500, 29: 500, 30: 500, 31: 500, 32: 500, 33: 500, 34: 500, 35: 500, 36: 500, 37: 500, 38: 500, 39: 500, 40: 500})
Correctness : Counter({1: 14452, 0: 5548})
cot_answer : Counter({6: 720, 20: 640, 5: 600, 4: 600, 15: 520, 10: 480, 3: 440, 8: 440, 2: 440, 16: 400, 50: 360, 18: 320, 9: 320, 40: 320, 21: 320, 45: 280, 25: 280, 30: 280, 12: 280, 60: 240, 14: 240, 80: 240, 48: 240, 7: 200, 26: 200, 70: 200, 75: 200, 36: 200, 32: 200, 1: 200, 17: 160, 100: 1

# CS based performance

In [802]:
def log_reg(features):
    coe = [-0.1, -5 , -1  , 3 , 2]
    lincomb = sum([coe[i]*features[i] for i in range(len(coe))]) -1.5
    return 1 / (1 + np.exp(-lincomb))
log_reg([5,0,0,1,1])

0.9525741268224334

In [803]:
df = hard
df_concate = hard_final
threshold = 0.36
warm_up_steps = 0

In [804]:
# df_concate['confidence_score']= cls_model.predict_proba(df_concate[feature_li])[:,1]
df_concate['confidence_score']=df_concate[feature_li].apply(log_reg,axis =1)
lists = [df_concate['confidence_score'].iloc[i:i+40].tolist() for i in range(0, len(df_concate), 40)]
df['confidence_score'] = lists

In [813]:
def normalize_cs(cs_li, threshold):
    cs_arr = np.array(cs_li)
    # normalized_cs = [(cs-threshold)/(1-threshold) if cs > threshold else (cs-threshold)/(threshold) for cs in cs_arr]
    normalized_cs = cs_arr-threshold
    # normalized_cs = [0.5 if cs > threshold else -0.5 for cs in cs_arr]
    return np.array(normalized_cs)

In [814]:
def stop_con1(individual_cs):
    cumulative_difference = (individual_cs[warm_up_steps:]).cumsum()
    stop_idx = np.argmax(cumulative_difference > 0.5)
    return stop_idx
def stop_con2(individual_cs):
    buffer = []
    for idx,cs in enumerate(individual_cs):
        if cs>0:
            buffer.append(idx)
        if len(buffer) == 3:
            return buffer[-1]

In [815]:
CS_Answer = []
CS_correctness = []
CS_steps = []
for row_idx in range(len(df)): 
    test_row = df.iloc[row_idx]
    individual_cs = normalize_cs(test_row['confidence_score'],threshold)
    # individual_cs = test_row['confidence_score'][warm_up_steps:] - threshold
    stop_idx = stop_con2(individual_cs)
    
    if stop_idx:
        num_of_steps = warm_up_steps+stop_idx
    else: num_of_steps = 40
    answers = test_row['CoT answers'][:num_of_steps]
    scores = individual_cs[:num_of_steps]
    print(scores)
    weighted_votes = Counter()
    for answer, score in zip(answers, scores):
        if score >0:
            weighted_votes[answer] += score
    # Find the answer with the highest total score
    if (len(weighted_votes)) == 0:
        for answer, score in zip(answers, scores):
            weighted_votes[answer] += score
    print(weighted_votes)
    result = max(weighted_votes, key=weighted_votes.get)
    CS_Answer.append(result)
    CS_correctness.append(1 if result == str(test_row['correct answer']) else 0)
    CS_steps.append(num_of_steps)
df['CS_Answer'] = CS_Answer
df['CS_correctness'] = CS_correctness
df['CS_steps'] = CS_steps
df

[-0.35999386 -0.27549978 -0.2765479  -0.25665826 -0.26141125 -0.25735374
  0.5836613  -0.26020923 -0.25693995 -0.25858628 -0.2589942  -0.26020923
  0.58270005 -0.25693995 -0.25693995 -0.25858628 -0.27692983]
Counter({'19735200': 0.583661301532343, '9867650': 0.5827000516297904})
[-0.35999386  0.57119954  0.58771084]
Counter({'3431580': 1.1589103765679512})
[-0.35999386  0.57010057  0.56764704 -0.27301875]
Counter({'14': 1.1377476103994615})
[-0.35999386  0.57250044 -0.26061544 -0.36       -0.24365666 -0.36
 -0.35999486 -0.36       -0.36       -0.35996206 -0.36       -0.36
 -0.36       -0.36       -0.35999283 -0.36       -0.35995704 -0.24895106
 -0.36       -0.36       -0.36       -0.36       -0.35999313 -0.36
 -0.36       -0.36       -0.36       -0.22575757 -0.22860639 -0.36
 -0.35987507 -0.26037416 -0.24043203 -0.23254058 -0.36       -0.24296715
 -0.36       -0.36       -0.36       -0.17757448]
Counter({'6887423': 0.5725004374171193})
[-0.24079708  0.58426734 -0.23648689 -0.2466492  -

,id,correct answer,CoT answers,Correctness,LEN,QUA_IM,DIF_IV,DIF_SUB,SIM_COT_BIGRAM,SIM_COT_AGG,...,SIM_AC_AGG,SIM_AC_PW,size_of_cot,SC_correctness,ES_correctness,ES_steps,confidence_score,CS_Answer,CS_correctness,CS_steps
0,0,-9867630,"[0, 9867650, 19735200, -9867630, 1973528, 1973...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 5, 5, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Challenging Math,"[0.0, 0.8827160493827161, 0.9006211180124224, ...","[0.5, 0.8827160493827161, 0.8963414634146342, ...",...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12...",0,0,40,"[6.144174602214718e-06, 0.08450021508887766, 0...",19735200,0,17
1,1,3431580,"[3431580, 3431580, 3431580, 3431580, 3431580, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[4, 4, 4, 4, 4, 4, 4, 3, 2, 4, 4, 4, 4, 2, 2, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, ...",Challenging Math,"[0.0, 0.8947368421052632, 0.6111111111111112, ...","[0.5, 0.8947368421052632, 0.6027397260273972, ...",...,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12...",1,1,5,"[6.144174602214718e-06, 0.9311995403759062, 0....",3431580,1,3
2,2,322886700,"[14, 14, 14, 107628900, 107628900, 14, 1076289...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, ...",Challenging Math,"[0.0, 0.9117647058823529, 1.0, 0.8676470588235...","[0.5, 0.9117647058823529, 0.948905109489051, 0...",...,"[0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, ...","[0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12...",0,0,28,"[6.144174602214718e-06, 0.9301005744685183, 0....",14,0,4
3,3,-6887448,"[6887423, 6887423, 6887398, 6887398, 167, 206,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4, 3, 3, 3, 3, 4, 5, 5, 3, 3, 5, 0, 3, 3, 3, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 5, 0, 5, 2, 5, 5, 2, 9, 10, 5, 9, 2,...",Challenging Math,"[0.0, 0.874251497005988, 0.6787564766839378, 1...","[0.5, 0.874251497005988, 0.7040816326530612, 0...",...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12...",0,0,40,"[6.144174602214718e-06, 0.9325004374171193, 0....",6887423,0,40
4,4,21459061,"[21459067, 21459067, 16104295, 21459060, 16119...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[6, 4, 1, 6, 5, 6, 5, 5, 4, 5, 1, 6, 3, 6, 5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Challenging Math,"[0.0, 0.6701570680628273, 0.4605263157894737, ...","[0.5, 0.6701570680628273, 0.4595744680851064, ...",...,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12...",0,0,40,"[0.11920292202211755, 0.9442673367004344, 0.12...",21459067,0,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,200,"[200, 200, 200, 200, 2400, 2400, 2400, 2400, 2...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, ...","[3, 3, 2, 2, 3, 3, 3, 3, 3, 2, 2, 2, 6, 3, 3, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Math,"[0.0, 0.944954128440367, 0.9166666666666666, 1...","[0.0033464254621424277, 0.017161154459243322, ...",...,"[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",1,0,9,"[0.18192694664438

In [816]:
print(df.ES_steps.mean(),df.CS_steps.mean())
print()
print(df.ES_correctness.value_counts(),df.CS_correctness.value_counts())
print()
print(df.ES_correctness.sum()/len(df),df.CS_correctness.sum()/len(df))

17.234 8.179

ES_correctness
1    634
0    366
Name: count, dtype: int64 CS_correctness
1    630
0    370
Name: count, dtype: int64

0.634 0.63


In [754]:

df_t07 = pd.read_json('../data/T0.7.jsonl', lines=True)

In [522]:
df_t07['generated_answer'].iloc[0][3]

"A: Janet's ducks lay 16 eggs per day. She eats 3 and uses 4 for muffins every day. So she has 16 - 3 - 4 = 9 eggs left to sell. She can sell the 9 eggs for 9 x 2 = $18 per day. The answer is $18."

In [1414]:
import re
ANS_RE = re.compile(r"#### (\-?[0-9\.\,]+)")
INVALID_ANS = "[invalid]"


def extract_answer(completion):
    match = ANS_RE.search(completion)
    if match:
        match_str = match.group(1).strip()
        match_str = match_str.replace(",", "")
        return match_str
    else:
        return INVALID_ANS
extract_answer(df_t07.iloc[0]['answer'])

'18'

In [1343]:
tem_1 = df_t07['generated_answer'].iloc[0][3]
number_list = re.findall(r"\d+\.?\d*", tem_1)
try:
    predict1 = number_list[-1].strip('.')
except:
    predict1 = -1000
predict1

'18'

In [937]:
number_list

['16', '3', '4', '16', '3', '4', '9', '9', '9', '2', '18', '18.']